# Execution tutorial

This tutorial covers the basics of executing a quantum circuit using Classiq directly through the Python SDK. It is also possible to use the [Classiq Platform](https://platform.classiq.io) to execute quantum algorithms.

For this, we will start by synthesizing the following example, from the [synthesis tutorial](https://docs.classiq.io/latest/explore/tutorials/basic_tutorials/the_classiq_tutorial/synthesis_tutorial/):

In [1]:
from classiq import *


@qfunc
def main(x: Output[QNum[3]], y: Output[QNum]) -> None:
    allocate(x)
    hadamard_transform(x)
    y |= x**2 + 1


qprog = synthesize(main)

This quantum program evaluates the function $y(x) = x^2 + 1$, for all integers $x \in [0,7]$. To execute a quantum program and save its results in the Python SDK, use `execute`:

In [2]:
results = execute(qprog)

The outputs of the quantum circuit can be obtained through the attributes `counts` and `parsed_counts` from `results.result_value()`.

In [3]:
print("Output of the executing using counts: ", results.result_value().counts)

print(
    "Output of the executing using parsed_counts: ",
    results.result_value().parsed_counts,
)

Output of the executing using counts:  {'000101010': 258, '011010101': 242, '000001000': 253, '100101110': 256, '001010011': 256, '000010001': 256, '010001100': 271, '110010111': 256}
Output of the executing using parsed_counts:  [{'x': 4, 'y': 17}: 271, {'x': 2, 'y': 5}: 258, {'x': 6, 'y': 37}: 256, {'x': 3, 'y': 10}: 256, {'x': 1, 'y': 2}: 256, {'x': 7, 'y': 50}: 256, {'x': 0, 'y': 1}: 253, {'x': 5, 'y': 26}: 242]


As we can see, the results from the execution are the same but differ in format: 

* `counts` will output a `dict` containing the bitstrings measured followed by its number of counts.
* `parsed_counts` will output a `list` of `dicts`, each containing the values of `x` and `y` followed by its number of counts.

These two ways of reading the results can be used in different.

## Backend selection

The backend of an execution is the hardware choice where the quantum program is executed. It can be classical hardware, i.e., a simulator, or real hardware. To select a specific backend, it is necessary to import its correct Backend Preferences from `classiq.execution`. Check the different [Cloud Providers](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) and their Backend Preferences for execution.

In this section we will explore two different examples for fixation: 

### First example: Execution using the state vector simulator from Classiq

Since Classiq provides its own state vector simulator backend, we will use `ClassiqBackendPreferences` to define it as the state vector simulator. This information is provided on the [Cloud Providers page](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/).

To define the quantum program's execution preferences, use `ExecutionPreferences` and then `set_quantum_program_execution_preferences`. In this example, we will perform a simulation with `num_shots=1` since the state vector simulator performs an exact simulation of the quantum program.

If no backend is defined on the preferences, then the [Classiq simulator](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/classiq-backends/) is selected.


In [2]:
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences

backend_preferences = ClassiqBackendPreferences(
    backend_name="simulator_statevector"
)  # Always check the Cloud Providers to correctly define the backend.

execution_preferences = ExecutionPreferences(
    num_shots=1, backend_preferences=backend_preferences
)

set_quantum_program_execution_preferences(qprog, execution_preferences)

QuantumProgram(outputs={<QuantumFormat.QASM: 'qasm'>: "// G...15];" (length=4146)}, qasm_version=<QasmVersion.V2: '2.0'>, version='0.79.1', interface_version='11', hardware_data=SynthesisHardwareData(basis_gates=['ry', 'x', 'u1', 'r', 'sxdg', 'z', 'id', 'sx', 'cz', 'u2', 's', 'y', 'tdg', 'p', 'cy', 'sdg', 'u', 'rx', 'rz', 't', 'h', 'cx'], connectivity_map=None, is_symmetric_connectivity=True, backend_data=None), initial_values=None, data=GeneratedCircuitData(width=16, circuit_parameters=[], qubit_mapping=QubitMapping(logical_inputs={}, logical_outputs={'x': (0, 1, 2), 'y': (3, 4, 5, 6, 7, 8)}, physical_inputs={}, physical_outputs={'x': (0, 1, 2), 'y': (3, 4, 5, 6, 7, 8)}), execution_data=None), model=ExecutionModel(version='0.79.1', interface_version='11', enums=[], types=[], qstructs=[], constants=[], classical_execution_code='', execution_preferences=ExecutionPreferences(noise_properties=None, random_seed=3442327117, backend_preferences=ClassiqBackendPreferences(backend_service_provi

Now, execute the quantum program using `execute`.

In [3]:
results_statevector = execute(qprog)

The outputs of the quantum program can be obtained through the attributes `state_vector` and `parsed_state_vector` from `results.result_value()`. Below, we can see the amplitudes for the state `'x':0, 'y':1`, which can be represented by the bitstring '0000000000001000':

In [4]:
print(
    "Amplitude of the state 0000000000001000: ",
    results_statevector.result_value().state_vector["0000000000001000"],
)

state = next(
    (
        s
        for s in results_statevector.result_value().parsed_state_vector
        if s.bitstring == "0000000000001000"
    ),
    None,
)
if state:
    print("Amplitude of the state {'x':0, 'y':1}: ", state.amplitude)

Amplitude of the state 0000000000001000:  (-0.35355339059327345-1.7801753908978445e-15j)
Amplitude of the state {'x':0, 'y':1}:  (-0.35355339059327345-1.7801753908978445e-15j)


The outputs from the execution obtained via statevector simulator will differ from the default simulator:

* `state_vector` will output a `dict` containing the bitstrings followed by its numerically evaluated amplitudes.
* `parsed_state_vector` will output a `list` of `SimulatedState`, each containing the values of `x` and `y` followed by its bitstrings and its numerically evaluated amplitudes.

### Second example: Execution on free access IBM Hardware

We will now execute our quantum program on `ibm_brisbane`, a quantum computer from IBM Quantum. For this, we need to check on the [Cloud Providers page](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) for the [IBM Quantum Backends](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/ibm-backends/). First, we need to do [hardware-aware synthesis](), i.e., synthesize the quantum program accordingly to the hardware that it will be executed. This can be done by specifying the backend using the `Preferences` function.

In [14]:
preferences = Preferences(
    backend_service_provider="IBM Quantum", backend_name="ibm_brisbane"
)
qprog = synthesize(main, preferences=preferences)

On the Cloud Providers page, we can see that an access token from an IBM Quantum account is required. Using this information, it is possible to set our execution preferences:

In [ ]:
from classiq.execution import (
    ExecutionPreferences,
    IBMBackendPreferences,
    IBMBackendProvider,
)

ibm_provider = IBMBackendProvider(hub="ibm-q", group="open", Project="main")
ibm_backend_preferences = IBMBackendPreferences(
    backend_name="ibm_brisbane", access_token="access_token", provider=ibm_provider
)

execution_preferences = ExecutionPreferences(
    num_shots=1000, backend_preferences=ibm_backend_preferences
)

qprog = set_quantum_program_execution_preferences(qprog, execution_preferences)

In [24]:
results_ibm_brisbane = execute(qprog).result_value()

Its outputs will formatted in the same way of the outputs of the Classiq simulator:

In [33]:
print(
    "Output of the executing using counts: ",
    results_ibm_brisbane.counts,
)

print(
    "Partial output of the executing using parsed_counts: ",
    results_ibm_brisbane.parsed_counts[:100],
)

Output of the executing using counts:  {'111011000': 2, '101101110': 4, '000110111': 3, '011101100': 4, '101001000': 3, '111010100': 1, '010101001': 5, '010010011': 3, '000000010': 3, '001011001': 4, '100011001': 4, '000011101': 2, '000101101': 4, '101110101': 4, '110001011': 3, '111101010': 2, '101111110': 2, '010010110': 5, '000000101': 3, '110100000': 3, '101001101': 3, '011010011': 4, '000000100': 2, '001000000': 2, '110010110': 3, '011100100': 3, '111011011': 4, '110100001': 5, '111100100': 3, '101011001': 2, '010001111': 1, '001111101': 6, '101000001': 2, '010100111': 4, '111101001': 2, '101110111': 4, '111011101': 7, '001011011': 4, '111010111': 2, '010100110': 2, '000001000': 3, '110110111': 2, '000101100': 5, '111101011': 1, '111100111': 1, '101001111': 3, '101100011': 2, '001000001': 1, '010010111': 5, '111110101': 4, '010101101': 4, '100101111': 5, '110100101': 5, '111011100': 2, '111001100': 2, '111010101': 7, '111000101': 1, '100110011': 4, '010000010': 4, '010111100': 4, 